In [1]:
import folium
from matplotlib.colors import Normalize, rgb2hex
import matplotlib.cm as cm
import urllib2
import json
import os
import datetime

In [2]:
zips3 = eval(open('kody_pocztowe3.dict').readline().strip())
zips2 = eval(open('kody_pocztowe2.dict').readline().strip())

In [4]:
import random
def add_companies_to_map(view, companies, coloUr = 'blue'):
    fg = folium.FeatureGroup(name = 'company locations ' + coloUr)
    random.seed(42)
    for i,company in enumerate(companies):
        lat, lng = None, None
        if company.postal == 'None':
            continue
        if zips3.get(company.postal[0:4], None) != None:
            lat, lng = zips3.get(company.postal[0:4])
        elif zips2.get(company.postal[0:2], None) != None:
            lat, lng = zips2.get(company.postal[0:2])
        else:
            continue
        lat += random.gauss(0.0, 0.0001)
        lng += random.gauss(0.0, 0.0001)
        fg.add_child(folium.Marker(location=[lat,lng], icon=folium.Icon(color=coloUr)))
    view.add_child(fg)

In [6]:
ceidg_files = map(lambda x: 'gotten/' + x, filter(lambda x: x.endswith('.parsed'), os.listdir('gotten/')))

In [7]:
class Company():
   def __init__(self, postal, date, name, codes):
       self.postal = postal
       self.date = date
       self.name = name
       self.codes = codes

In [8]:
def read_company(row):
    kod,czas,naz,dzi = row.strip().split(',', 3)
    dzi = dzi.split(',')
    czas = datetime.date(*map(int, czas.split('-')))
    return Company(kod, czas, naz, dzi)

In [9]:
from collections import defaultdict
def groupBy(lista, fun):
    ret = defaultdict(lambda: [])
    for el in lista:
        ret[fun(el)].append(el)
    return ret

In [13]:
def ok(comp):
    return comp.name.find('kebab') > -1 or comp.name.find('kebap') > -1

In [ ]:
import subprocess

prev_companies = None
for year in range(1990, 2017):
    view = folium.Map(location=[52, 19], zoom_start=6)
    companies = []
    for file in ceidg_files:
        if int(file[len('gotten/ceidg'):len('gotten/ceidg')+4]) < year:
            with open(file) as inF:
                for row in inF:
                    if not row.startswith('None'):
                        comp = read_company(row)
                        if ok(comp):
                            companies.append(comp)
    add_companies_to_map(view, companies)
    companies = []
    for file in ceidg_files:
        if int(file[len('gotten/ceidg'):len('gotten/ceidg')+4]) == year:
            with open(file) as inF:
                for row in inF:
                    if not row.startswith('None'):
                        comp = read_company(row)
                        if ok(comp):
                            companies.append(comp)
    add_companies_to_map(view, companies, 'red')
    url = 'temporary/{}.html'.format(str(year))
    png = 'temporary/{}.png'.format(str(year))
    gif = 'temporary/{}.gif'.format(str(year))
    view.save(url)
    subprocess.call(['/home/jkmilczek/cutycapt/CutyCapt/CutyCapt', '--url={}'.format(url), '--out={}'.format(png), '--delay=2000'])
    os.system('convert {} -pointsize 41 -draw \'text 10,50 "Year {}"\' {}'.format(png, year, gif))

In [ ]:
with open('out.gif', 'w') as outF:
    os.system('gifsicle --delay=50 temporary/*.gif > kebab.gif')